<a href="https://colab.research.google.com/github/ClauCll/Curso_ML_A_Z/blob/master/Regresi%C3%B3n/Regresion_Bajas_Antiguedad_v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cuantas serán las bajas voluntarias de los clientes fijos para un periodo x. (Agosto - Diciembre 2020)
 

In [1]:
import pandas as pd
import numpy as np

#Gráfico
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt 
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

import gspread
from oauth2client.client import GoogleCredentials 
from google.colab import auth
# Autoriza Claudia
auth.authenticate_user()
gc = gspread.authorize (GoogleCredentials.get_application_default())

### leemos datos

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/My Drive/Dataset/M_BAJAS.txt',delimiter='~')

In [ ]:
data.head(5)

In [5]:
encabezados= ['Año','Mes','Fecha','Antiguedad','Cant_Bajas']
data.columns=encabezados

In [6]:
data.Fecha = pd.to_datetime(data.Fecha)

In [ ]:
data.head(5)

In [11]:
data.dtypes

Año                    int64
Mes                    int64
Fecha         datetime64[ns]
Antiguedad          category
Cant_Bajas             int64
dtype: object

In [ ]:
 #validamos datos faltantes
 print(pd.isnull(data).sum())

In [ ]:
data.describe(include='all')

In [ ]:
data.info 

In [8]:
#Agrupamos la columna antiguedad
#1 : 1 TRIMESTRE (0-93 DIAS)
#2 : 2 TRIMESTRE (94-186 DIAS)
#3 : 3 TRIMESTRE (187-279 DIAS)
#4 : 4 TRIMESTRE (280-365 DIAS)
#5 : A 2 AÑOS (366-744 DIAS) 
#6 : A 4 AÑOS (745-1488 DIAS)
#7 : Mayor 4 años (Máximo DIAS 17101) 
bins=[0,93,186,279,365,744,1488,17110]
names=['1','2','3','4','5','6','7']
data['Antiguedad'] = pd.cut(data['Antiguedad'],bins,labels=names)

In [ ]:
 #validamos datos faltantes
 print(pd.isnull(data).sum())

In [ ]:
data.dropna(subset = ['Antiguedad'] , axis=0, inplace=True)
data['Antiguedad'].astype(str).astype(int)

In [18]:
data=data.drop(['Fecha'],axis=1)

In [ ]:
data.set_index( 'Fecha' ).groupby ( 'Año' ) [ "Mes" ].remuestrear( "M" ).suma()

In [ ]:
data3

In [183]:
data= pd.get_dummies(data,columns=['Antiguedad'])

In [ ]:
data

In [19]:
data.corr(method ='pearson')

,Año,Mes,Cant_Bajas
Año,1.000000,-0.318232,0.029800
Mes,-0.318232,1.000000,-0.003926
Cant_Bajas,0.029800,-0.003926,1.000000


In [20]:
x= np.array(data.drop(['Cant_Bajas'],axis=1))
y= np.array(data['Cant_Bajas'])

In [21]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [28]:
from sklearn.svm import SVR

svr_ = SVR(kernel='rbf',C=1.0,epsilon=0.2)
svr_.fit(x_train, y_train)
y_pred= svr_.predict(x_test)

In [29]:
svr_.score(x_train,y_train)

-0.102827173078758

In [37]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree = 3)
x_poly = poly_reg.fit_transform(x_train)

In [ ]:
print(pd.DataFrame(x_poly))

In [39]:
from sklearn.linear_model import LinearRegression
lineal_poly = LinearRegression()
lineal_poly.fit(x_poly,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [40]:
lineal_poly.score(x_poly,y_train)

0.06117165339805852